In [ ]:

# Instalación (Colab)
!pip install SpeechRecognition pydub fpdf

import os
import speech_recognition as sr
from pydub import AudioSegment
from fpdf import FPDF

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 54.7 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=4126f65c41ef4dff76a9ecf304871fc6aabc721adc2bd0201afe84eb30b66472
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [ ]:

audio_path = "/content/audio.m4a"
segment_dir = "/content/segmentos"
os.makedirs(segment_dir, exist_ok=True)

audio = AudioSegment.from_file(audio_path)
segment_length = 60000  # 60s

segments = []
for i, start in enumerate(range(0, len(audio), segment_length)):
    segment = audio[start:start+segment_length]
    filename = f"segmento_{i:03d}.wav"
    path = os.path.join(segment_dir, filename)
    segment.export(path, format="wav")
    segments.append(filename)


In [ ]:

recognizer = sr.Recognizer()
files = sorted(
    os.listdir(segment_dir),
    key=lambda x: int(x.split("_")[1].split(".")[0])
)

transcripcion = ""

for file in files:
    with sr.AudioFile(os.path.join(segment_dir, file)) as source:
        audio_data = recognizer.record(source)
        try:
            texto = recognizer.recognize_google(audio_data, language="es-ES")
            transcripcion += texto + " "
        except sr.UnknownValueError:
            print(f"No se entendió {file}")


In [ ]:

with open("/content/transcripcion.txt", "w", encoding="utf-8") as f:
    f.write(transcripcion)


In [ ]:

pdf = FPDF()
pdf.add_page()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.set_font("Arial", size=12)
pdf.multi_cell(0, 8, transcripcion)

pdf.output("/content/transcripcion.pdf")


''